<a href="https://colab.research.google.com/github/skauntey/movie-sentiment-analysis/blob/tf_tokenizer/Categorical_Conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir ~/.kaggle
!echo '{"username":"kaunteyshah","key":"e2678a9d0e44e057000b9daf059111b5"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os, shutil
import kaggle

#### 1. Downloading Kaggle files

In [ ]:
kaggle.api.authenticate()
kaggle.api.competition_download_files('sentiment-analysis-on-movie-reviews', path= str(os.getcwd())+"/dataset/", force = True)

#### 2. Unzipping Kaggle files

In [ ]:
import zipfile
ziped_file = r'dataset/sentiment-analysis-on-movie-reviews.zip'
folder_path = os.path.join(str(os.getcwd()), 'dataset/')
for files in os.listdir(folder_path):
  path = os.path.join(os.getcwd()+'/'+'dataset/',files)
  if path.split('.')[-1] != "zip":
    try:
      shutil.rmtree(path)
    except:
      raise
with zipfile.ZipFile(ziped_file) as zip_file:
  for member in zip_file.namelist():
      if member.split('.')[-1] == "zip":
          fdir = member.split('.')[0]
          zip_file.extract(member, path= os.path.join('dataset/'+fdir))
          # extracting individual train and test files in their respective folders
          zippedfile_name = os.listdir(os.path.join(str(os.getcwd()+'/dataset/'+fdir+'/')))
          zippedfile = os.path.join(str(os.getcwd())+'/dataset/'+ fdir, zippedfile_name[0])
          with zipfile.ZipFile(zippedfile, mode='r') as tsv_zip:
              tsv_zip.extractall(path = os.path.join('dataset/'+fdir+'/'))
                
          os.remove(zippedfile)
      else:
        continue
            
os.remove(ziped_file)


#### 3. Testing files

In [ ]:
tsvfile = 'dataset/train/train.tsv'
# read the data
tsv_read = pd.read_csv(tsvfile, sep="\t")

original_dataset = pd.DataFrame({ 
    'DATA_COLUMN': tsv_read['Phrase'], 
    'LABEL_COLUMN': tsv_read['Sentiment']})

pdf_data = original_dataset['DATA_COLUMN']
pdf_labels = original_dataset['LABEL_COLUMN']

In [ ]:
train_split = 0.9
val_split = 0.1
test_split = 0

training_mark = int(abs(len(original_dataset) * train_split))
# validation_mark = np.add(training_mark, int(abs(len(pdf)) * val_split))

print(training_mark)
# print(validation_mark)

training_set = original_dataset.loc[:training_mark,:]
validation_set = original_dataset.loc[training_mark:,:]

train_data = training_set['DATA_COLUMN']
train_labels = training_set['LABEL_COLUMN']

val_data = validation_set['DATA_COLUMN']
val_labels = validation_set['LABEL_COLUMN']

print(train_labels.head())
print(val_labels.head())

print(train_data.head())
print(train_labels.head())

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding
from tensorflow.keras.models import Sequential

In [ ]:
df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))



def clean_text(text):
  """
    text: a string
    return: modified initial string
  """
  text = text.lower() # lowercase text
  text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
  text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
  text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
  text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
  return text

In [ ]:

def tokenizing_dataset(original_dataset):
  tokenizer = Tokenizer(num_words = 15290, oov_token='<00V>')
  tokenizer.fit_on_texts(original_dataset)
  word_index = tokenizer.word_index
  return word_index, tokenizer

def Sequencing_dataset(original_dataset, dataset):
  _, pretrained_tokenizer = tokenizing_dataset(original_dataset)
  sequences = pretrained_tokenizer.texts_to_sequences(dataset)
  padded_sentences = pad_sequences(sequences, maxlen= 49, truncating ='post', padding='post')
  return padded_sentences, sequences

padded_train, train_sequences = Sequencing_dataset(pdf_data, train_data)
padded_val, val_sequences = Sequencing_dataset(pdf_data, val_data)
word_index_pdf, pdf_tokenizer = tokenizing_dataset(pdf_data)


print(len(word_index_pdf))
print(max([len(x) for x in train_sequences]))
print(padded_val[0])
print(padded_train[0])
print(train_data[0])


In [ ]:
def decoding_sequences(original_dataset, sequences):
  word_index, _ = tokenizing_dataset(original_dataset)
  reversed_index = dict(map(reversed, word_index.items()))
  for letter in range(len(sequences)):
      reversed_index.get(letter) 
  reversed_sequence = [reversed_index.get(letter) for letter in sequences]
  return (' '.join(reversed_sequence))
    
print(decoding_sequences(pdf_data, train_sequences[0]))
print(train_data[0])

In [ ]:
vocabulary_size = len(word_index_pdf) + 1
embedding_dim = 128
max_length = 49
trunc_type = 'post'
oov_tok = '<00V>'

"""model = tf.keras.Sequential([
                             
    tf.keras.layers.Embedding(vocabulary_size, embedding_dim, input_length = max_length),
    #tf.keras.layers.Flatten(),
    #tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.LSTM(64, return_sequences=False, dropout = 0.2),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(5, activation ='sigmoid')
])"""

model = Sequential()
model.add(Embedding(len(word_index_pdf)+1, embedding_dim, input_length=max_length))

model.add(LSTM(64, return_sequences=True, dropout=0.2))
model.add(LSTM(30))

model.add(Dense(60, activation='relu'))
model.add(Dense(5, activation='softmax'))

optimizer = tf.keras.optimizers.Adam(lr=0.001)
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer=optimizer, metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])
print(model.summary())

In [ ]:
num_epochs = 30
batch_size = 10000
history = model.fit(padded_train, train_labels, batch_size=batch_size, epochs = num_epochs, validation_data=(padded_val, val_labels), validation_batch_size=batch_size)

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)